# Automatic import of Hamburg drinking fountains data to Wikidata
The following script downloads fountain data from Open Data swiss as per https://github.com/water-fountains/proximap/issues/307, compares it to existing fountains in Wikidata for the same region, and creates Wikidata Quickstatement commands to complete the entries in Wikidata. New entities are created if no matching fountains are found.

## Initialize environment

In [1]:
from datetime import datetime as dt
dtFmt = "%y%m%d_%H%M%S"
print (dt.now().strftime(dtFmt))
import pandas as pd
import io
import numpy as np
from urllib.request import urlopen
import json
from math import *
from platform import python_version
print("Python v "+python_version())
#https://github.com/paulhoule/gastrodon/issues/7 
from gastrodon import RemoteEndpoint,QName,ttl,URIRef,inline
from matplotlib import pyplot


191123_122915
Python v 3.6.5


In [13]:
#@prefix wikibase: <wikibase: <http://wikiba.se/ontology#> .
prefixes=inline("""
   @prefix wd: <http://www.wikidata.org/entity/> .
   @prefix wdt: <http://www.wikidata.org/prop/direct/> .
   @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
   @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
""").graph
endpoint=RemoteEndpoint(
   "https://query.wikidata.org/sparql"
   ,prefixes=prefixes
)

## Load data

In [21]:
df = pd.read_csv("osmFountainsHamburg191123_123112.csv")

In [22]:
df.head()

,type,id,lat,lon,tags
0,node,257690357,53.551760,10.020930,{'amenity': 'fountain'}
1,node,268192970,53.549850,9.935483,"{'amenity': 'fountain', 'image': 'http://commo..."
2,node,276284733,53.545274,9.971903,"{'amenity': 'drinking_water', 'drinking_water:..."
3,node,292561393,53.570260,9.980357,{'amenity': 'fountain'}
4,node,297703374,53.580721,9.704032,{'amenity': 'fountain'}


### Rename columns to make them easier to work with

In [23]:
# remove not needed columns
df = df.drop(columns=['type','id','tags'])

In [24]:
# rename columns
df = df.rename(index=str, columns=
               {"lat": "Y"})

In [25]:
# rename columns
df = df.rename(index=str, columns=
               {"lon": "X"})

In [46]:
df.head()
len(df)

127

## Identify already existing fountains
### Query fountains from Wikidata

In [27]:
# Find the geographic extent of the data

buffer = 0.0003  # in degrees, corresponds to about 20-30 meters)
bounds = {
    'minX': df['X'].min() - buffer,
    'minY': df['Y'].min() - buffer,
    'maxX': df['X'].max() + buffer,
    'maxY': df['Y'].max() + buffer
}
print("bounds: ")
for key,value in bounds.items():
    print(key+ ": "+str(value))

bounds: 
minX: 9.663358500000001
minY: 53.425859700000004
maxX: 10.2964728
maxY: 53.731296900000004


In [28]:
# Query fountains (both water wells and fountains) from Wikidata within bounding box found above

query_string = """SELECT ?place ?placeLabel ?location ?date ?catalog_code ?catalogLabel ?operator ?water_supply_type
WHERE
{{
  # Enter coordinates
  SERVICE wikibase:box {{
    ?place wdt:P625 ?location .
    bd:serviceParam wikibase:cornerWest "Point({minX} {minY})"^^geo:wktLiteral.
    bd:serviceParam wikibase:cornerEast "Point({maxX} {maxY})"^^geo:wktLiteral.
  }} .
  # Is a water well or fountain or subclass of fountain
  FILTER (EXISTS {{ ?place wdt:P31/wdt:P279* wd:Q43483 }} || EXISTS {{ ?place wdt:P31/wdt:P279* wd:Q483453 }}).
  SERVICE wikibase:label {{
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],de" .
  }} 
  OPTIONAL {{ ?place p:P528 ?catalog_code.
            ?catalog_code pq:P972 ?catalog.}}
  OPTIONAL {{ ?place wdt:P571 ?date.}}
  OPTIONAL {{ ?place wdt:P5623 ?water_supply_type}}
  OPTIONAL {{ ?place wdt:P137 ?operator.}}
}}
  """.format(**bounds)

query_string

'SELECT ?place ?placeLabel ?location ?date ?catalog_code ?catalogLabel ?operator ?water_supply_type\nWHERE\n{\n  # Enter coordinates\n  SERVICE wikibase:box {\n    ?place wdt:P625 ?location .\n    bd:serviceParam wikibase:cornerWest "Point(9.663358500000001 53.425859700000004)"^^geo:wktLiteral.\n    bd:serviceParam wikibase:cornerEast "Point(10.2964728 53.731296900000004)"^^geo:wktLiteral.\n  } .\n  # Is a water well or fountain or subclass of fountain\n  FILTER (EXISTS { ?place wdt:P31/wdt:P279* wd:Q43483 } || EXISTS { ?place wdt:P31/wdt:P279* wd:Q483453 }).\n  SERVICE wikibase:label {\n    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],de" .\n  } \n  OPTIONAL { ?place p:P528 ?catalog_code.\n            ?catalog_code pq:P972 ?catalog.}\n  OPTIONAL { ?place wdt:P571 ?date.}\n  OPTIONAL { ?place wdt:P5623 ?water_supply_type}\n  OPTIONAL { ?place wdt:P137 ?operator.}\n}\n  '

In [29]:
# Perform query
query_result = endpoint.select(query_string)

In [31]:
#print(query_string)
print("\n\nTotal number of rows incl. duplicates "+str(query_result.size))



Total number of rows incl. duplicates 48


### Tidy up data

In [47]:
# Extract coordinates from Wikidata results

query_result['X'] = query_result['location'].apply(lambda l:float(l.split('(')[1].split(' ')[0]))
query_result['Y'] = query_result['location'].apply(lambda l:float(l.split(' ')[1].split(')')[0]))
query_result.head(100)

,place,placeLabel,location,date,catalog_code,catalogLabel,operator,water_supply_type,X,Y
0,wd:Q814528,Behnbrunnen,Point(9.94194444 53.54805556),None,None,None,None,None,9.941944,53.548056
1,wd:Q1454814,Minervabrunnen,Point(9.95166667 53.54583333),None,None,None,None,None,9.951667,53.545833
2,wd:Q2651496,Alsterfontäne,Point(9.99495278 53.55461667),None,None,None,None,None,9.994953,53.554617
3,wd:Q2358953,Stuhlmannbrunnen,Point(9.93527778 53.54972222),None,None,None,None,None,9.935278,53.549722
4,wd:Q18632417,Vierländerin-Brunnen,Point(9.99014 53.54766),None,None,None,None,None,9.990140,53.547660
5,wd:Q19285890,Hygieia-Brunnen,Point(9.991934 53.550258),None,None,None,None,None,9.991934,53.550258


In [48]:
# show duplicates
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html#pandas.DataFrame.duplicated

ids = query_result['placeLabel']
duplicates = query_result[ids.isin(ids[ids.duplicated(keep=False)])]
dupli = duplicates.drop(columns=['catalog_code', 'catalogLabel', 'water_supply_type', 'date', 'catalogLabel', 'location', 'operator'])
dupli.sort_values(by=['placeLabel'],inplace=True)
print ("Duplicates: "+str(dupli.size)+"\n\n") #shows rather the number of elements than rows
print(dupli.to_string())

Duplicates: 0


Empty DataFrame
Columns: [place, placeLabel, X, Y]
Index: []


In [34]:
def createMerge(dupli, extraText):
    #Create MERGE Command
    linesDup = []
    prevQ= ""
    prevX=0
    prevY=0
    for index, row in dupli.iterrows():
        
        # either create new or edit existing entity
        if row['X'] == prevX:
            if row['Y'] == prevY:
                lineDup = "MERGE\t"+prevQ[3:]+"\t"+row['place'][3:]+"\n"
                linesDup.append(lineDup)
            else:
                prevX=row['X']
                prevY=row['Y']
                prevQ=row['place']
        else:
            prevX=row['X']
            prevY=row['Y']
            prevQ=row['place']
            
    print("Merge commands"+extraText+" total: "+str(len(linesDup))+"\n\n")
    print(linesDup)
    
createMerge(dupli, "")    

Merge commands total: 0


[]


In [19]:
#write MERGE command to File
quickStatDupliFileName = "quickstatement_commands_Hamburg_drink_DUPLI_"+dt.now().strftime(dtFmt)+".txt"
with io.open(quickStatDupliFileName, "w", encoding='utf8') as f:
    f.writelines(linesDup)
print("wrote '"+quickStatDupliFileName+"' with "+str(len(linesDup))+" lines")

wrote 'quickstatement_commands_BS_drink_DUPLI_191101_070225.txt' with 2 lines


In [20]:
# show duplicates STRICT
#idsS = query_result['placeLabel','X','Y'] #this does not work yet
#duplicatesS = query_result[ids.isin(ids[ids.duplicated(keep=False)])]
#dupliS = duplicatesS.drop(columns=['catalog_code', 'catalogLabel', 'water_supply_type', 'date', 'catalogLabel', 'location', 'operator'])
#dupliS.sort_values(by=['placeLabel'],inplace=True)
#print(dupliS.to_string())
#createMerge(dupliS, " - strict")  

In [35]:
# remove duplicate entries

# duplicate entries are caused when e.g. a fountain has catalog codes from two catalogs
query_result = query_result.drop_duplicates('place')
print("\n\nTotal number of rows without duplicates "+str(query_result.size))



Total number of rows without duplicates 60


### Compute distances between fountains

In [49]:
# helper function to compute distances on the globe, returns distances in meters
def spherical_dist(pos1, pos2, r=6371000):
    pos1 = pos1 * np.pi / 180
    pos2 = pos2 * np.pi / 180
    cos_lat1 = np.cos(pos1[..., 0])
    cos_lat2 = np.cos(pos2[..., 0])
    cos_lat_d = np.cos(pos1[..., 0] - pos2[..., 0])
    cos_lon_d = np.cos(pos1[..., 1] - pos2[..., 1])
    return r * np.arccos(cos_lat_d - cos_lat1 * cos_lat2 * (1 - cos_lon_d))


# compute distances from each ODZ fountain to each Wikidata fountain
distances = spherical_dist(df[['X','Y']].values[:, None], query_result[['X','Y']].values)

### Identify nearest and second nearest matches for each osm hamburg fountain

In [50]:
# indexes of nearest fountains
nearest_idx = np.argmin(distances, axis=1).tolist()

# QID of nearest fountains
df['nearest_qid'] = query_result.iloc[nearest_idx]['place'].apply(lambda id:id[3:]).tolist()

# distance to nearest fountain
df['nearest_distance'] = np.min(distances, axis=1).tolist()


# then remove nearest
i_line=0
for i_col in nearest_idx:
    distances[i_line, i_col] = 100000
    i_line += 1
# find distance to second nearest
df['2nd_nearest_distance'] = np.min(distances, axis=1).tolist()

df.head(100)

,Y,X,nearest_qid,nearest_distance,2nd_nearest_distance,nearest_has_label_de,nearest_has_date,nearest_has_operator,nearest_has_code,nearest_has_water_type,match_found
0,53.551760,10.020930,Q2651496,2905.399310,3228.379375,True,False,False,False,False,no match
1,53.549850,9.935483,Q2358953,26.737057,744.874890,True,False,False,False,False,no match
2,53.545274,9.971903,Q18632417,2044.624115,2251.010283,True,False,False,False,False,no match
3,53.570260,9.980357,Q2651496,2359.821387,2540.691194,True,False,False,False,False,no match
4,53.580721,9.704032,Q2358953,25936.658656,26695.614011,True,False,False,False,False,no match
...,...,...,...,...,...,...,...,...,...,...,...
95,53.442080,9.885816,Q2358953,13010.319061,13179.440722,True,False,False,False,False,no match
96,53.556267,9.978353,Q18632417,1614.366015,1647.264243,True,False,False,False,False,no match
97,53.442161,9.885998,Q2358953,12993.698812,13162.018023,True,False,False,False,False,no match
98,53.442489,9.884875,Q2358953,13014.439064,13190.010172,True,False,False,False,False,no match


### Find out what information already exists for the nearest fountains

In [51]:
# does nearest have label in german?
df['nearest_has_label_de'] = (query_result.iloc[nearest_idx]['place'].apply(lambda p:p[3:]) != query_result.iloc[nearest_idx]['placeLabel']).tolist()

# does nearest have date?
df['nearest_has_date'] = query_result.iloc[nearest_idx]['date'].apply(lambda d:d is not None).tolist()

# does nearest have operator?
df['nearest_has_operator'] = query_result.iloc[nearest_idx]['operator'].apply(lambda id:id is not None).tolist()

# does nearest have catalog code?
df['nearest_has_code'] = query_result.iloc[nearest_idx]['catalog_code'].apply(lambda id:id is not None).tolist()

# does nearest have water type?
df['nearest_has_water_type'] = query_result.iloc[nearest_idx]['water_supply_type'].apply(lambda id:id is not None).tolist()

### Decide on whether nearest fountain should be considered a match

In [52]:
# The nearest fountain is a match if: 
# - no further than x m away
# - 2nd nearest fountain at nearest least ratio_min further away than the nearest fountain
def validate_proposal(qid, d1, d2, dmax=10, ratio_min=0.5):
    
    if d1 == 0 or (d1<=dmax and d2/d1-1 >= ratio_min):
        return 'match'
    elif d1<=dmax and d2/d1-1 < ratio_min:
        return 'unclear'
    else:
        return 'no match'
    
for index, row in df.iterrows():
    df.loc[index, 'match_found'] = validate_proposal(
        row['nearest_qid'], 
        row['nearest_distance'], 
        row['2nd_nearest_distance'],
        dmax=15
    )
dffinal = df.drop(columns=['nearest_distance', '2nd_nearest_distance'])

In [53]:
dffinal.head()

,Y,X,nearest_qid,nearest_has_label_de,nearest_has_date,nearest_has_operator,nearest_has_code,nearest_has_water_type,match_found
0,53.551760,10.020930,Q2651496,True,False,False,False,False,no match
1,53.549850,9.935483,Q2358953,True,False,False,False,False,no match
2,53.545274,9.971903,Q18632417,True,False,False,False,False,no match
3,53.570260,9.980357,Q2651496,True,False,False,False,False,no match
4,53.580721,9.704032,Q2358953,True,False,False,False,False,no match


In [54]:
print("\n\nTotal number of rows "+str(dffinal.size))



Total number of rows 1143


## Create Quickstatement commands from data
### Helper functions to format content according to Quickstatements v1 syntax

In [55]:
def process_coordinates(x, y):
    # format geographic coordinates
    return '@{1:1.8f}/{0:1.8f}'.format(x,y)


def process_year(date):
    # format date
    if np.isnan(date):
        return ''
    else:
        return '+{0:4d}-00-00T00:00:00Z/9'.format(int(date))

    
fountain_type_map = {
    'öffentlicher Brunnen': 'Q53628296',
    'Notwasserbrunnen': 'Q53628522',
    'privater Brunnen': 'Q53629707',
    'Brunnen in städtischer Liegenschaft': 'Q53628618',
    'Brunnen des Verschönerungsvereins': 'Q53628761',
    'Brunnen mit eigener Versorgung': 'Q53630002'
}

water_type_map = {
    'Verteilnetz': 'Q53633635',
    'Quellwasser': 'Q1881858',
    'eigene Versorgung': 'Q53634173',
    'Grundwasser': 'Q161598'
}

def process_fountain_type(type):
    # translate fountain types to wikidata values
    return fountain_type_map[type]


def process_water_type(type):
    # translate water types to wikidata values
    return water_type_map[type]


def process_label_de(text):
    # process German language labels
    if text is None:
        return ''
    elif 'brunnen' in text.lower():
        return '"{}"'.format(text)
    else:
        return '"Brunnen ({})"'.format(text)
    

def createline(lines, item, prop, value, qualifiers=[]):
    # general function to create Quickstatement v1 commands
    if value != '' and value != '""':
        statement = '{}\t{}\t{}'.format(item, prop, value)
        if len(qualifiers):
            # append qualifiers if applicable
            for q in qualifiers:
                statement += '\t{}\t{}'.format(q['prop'], q['value'])
        statement += '\n'
        lines.append(statement)
    return lines

### Create statements, taking care not to overwrite existing data

In [57]:
# initialize command storage list
lines = []

for index, row in dffinal.iterrows():
    
    # either create new or edit existing entity
    if row['match_found'] == 'no match':
        # create a new fountain
        lines.append('CREATE\n')
        item = 'LAST'
    elif row['match_found'] == 'unclear':
        print('unclear match')
        print(row)
        continue
    elif row['match_found'] == 'match':
        # update existing fountain
        item = row['nearest_qid']
        
        
    # Add this basic information only if creating a new entity
    if item == 'LAST':
        # instance of drinking fountain
        lines = createline(lines, item, 'P31', 'Q483453')

        # coordinates
        lines = createline(lines, item, 'P625', process_coordinates(row['X'], row['Y']))
        
        # TODO put reference "https://opendata.swiss/dataset/bade-trinkwasser-und-zierbrunnen-in-basel"
        
        
    # For other properties, add information if the entity is new or if property does not yet exist
    
    # label in german
    #if item == 'LAST' or not row['nearest_has_label_de']:
    #    lines = createline(lines, item, 'Lde', process_label_de(row['label_de']))
    
    # creation date
    #if item == 'LAST' or not row['nearest_has_date']:
    #    lines = createline(lines, item, 'P571', process_year(row['date']))

    # operated by  Hamburg Wasser
    # lines = createline(lines, item, 'P137', 'Q1572943')
            
    # catalog number can always be added (it is hard to check for)
    #lines = createline(lines, item, 'P528', '"{}"'.format(row['operator_id']), [{
    #    'prop': 'P972',
    #    'value': 'Q53629101'
    #}])

# Write commands to file

In [58]:
quickStatFileName = "quickstatement_commands_Hamburg_fountain_"+dt.now().strftime(dtFmt)+".txt"
with io.open(quickStatFileName, "w", encoding='utf8') as f:
    f.writelines(lines)
print("wrote '"+quickStatFileName+"' with "+str(len(lines))+" lines")

wrote 'quickstatement_commands_Hamburg_fountain_191123_131209.txt' with 372 lines


# Import into Wikidata
- Go to https://tools.wmflabs.org/wikidata-todo/quick_statements.php.
- Authenticate yourself with your Wikidata account.
- Copy and paste the contents of quickstatement_commands*.txt into the blank field, and run the commands

see ../20191030_1600_import.png

...
58. Processing Q72935495 (Q72935495 Lde "Brunnen (Seelöwe-Planschbecken )")
59. Processing Q72935495 (Q72935495 P137 Q27229237)

All done!.